# Tigrigna Keyboard Input

This notebook creates a custom Tigrigna keyboard widget that can be used to input Tigrigna text.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import re

## Define the Tigrigna Characters Layout

In [ ]:
# Define the Tigrigna keyboard layout
# First row (numbers and symbols)
row1 = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

# Second row (first row of Tigrigna characters)
row2 = ['ሀ', 'ለ', 'ሐ', 'መ', 'ሠ', 'ረ', 'ሰ', 'ሸ', 'ቀ', 'በ']

# Third row (second row of Tigrigna characters)
row3 = ['ተ', 'ቸ', 'ኀ', 'ነ', 'ኘ', 'አ', 'ከ', 'ኸ', 'ወ', 'ዐ']

# Fourth row (third row of Tigrigna characters)
row4 = ['ዘ', 'ዠ', 'የ', 'ደ', 'ጀ', 'ገ', 'ጠ', 'ጨ', 'ጰ', 'ጸ']

# Fifth row (fourth row of Tigrigna characters)
row5 = ['ፀ', 'ፈ', 'ፐ', ' ', 'ቨ', 'ቦ', 'ቱ', 'ሙ', 'ሉ']

# Create row structure
keyboard_layout = [row1, row2, row3, row4, row5]

## Create Vowel Variants

In [ ]:
# Define the different vowel forms for each consonant base
# The index corresponds to different vowel forms in Tigrigna
# Format: 0:ə, 1:u, 2:i, 3:a, 4:e, 5:ə̄, 6:o

def generate_variants(base_char):
    """Generate vowel variants for a Tigrigna character"""
    # Define the main character ranges
    if base_char == ' ':
        return [' ']
    
    # Try to find the base character's Unicode code point
    base_code = ord(base_char)
    
    # The standard pattern in Ethiopic script is that vowel variants follow the base character
    # Base+0: 'ə', Base+1: 'u', Base+2: 'i', Base+3: 'a', Base+4: 'e', Base+5: 'ə̄', Base+6: 'o'
    variants = []
    for i in range(7):
        try:
            variant = chr(base_code + i)
            variants.append(variant)
        except:
            variants.append(base_char)  # Fallback to the base character if variant doesn't exist
    
    return variants

## Create the Keyboard UI

In [ ]:
def create_keyboard_ui():
    """Create a Tigrigna keyboard UI using ipywidgets"""
    # Create the text area for input
    text_area = widgets.Textarea(
        placeholder='Type Tigrigna text here...',
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    # Create a container for the vowel variant buttons (will be shown when a consonant is selected)
    variants_container = widgets.HBox([], layout=widgets.Layout(width='100%', height='40px'))
    
    # Keep track of the currently selected base character
    current_base = [None]
    
    # Container to hold all the buttons
    keyboard_container = widgets.VBox([])
    rows = []
    
    # Special buttons
    backspace_button = widgets.Button(
        description='⌫',
        layout=widgets.Layout(width='40px', height='40px')
    )
    
    space_button = widgets.Button(
        description='Space',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # Create buttons for each character in the layout
    for row in keyboard_layout:
        row_buttons = []
        for char in row:
            if char == ' ':
                # Add a space button
                row_buttons.append(space_button)
            else:
                button = widgets.Button(
                    description=char,
                    layout=widgets.Layout(width='40px', height='40px'),
                    style=widgets.ButtonStyle(font_weight='bold')
                )
                row_buttons.append(button)
        
        # Add buttons for this row
        rows.append(widgets.HBox(row_buttons, layout=widgets.Layout(width='100%')))
    
    # Add a row for special buttons
    special_row = widgets.HBox([backspace_button], layout=widgets.Layout(width='100%'))
    rows.append(special_row)
    
    # Function to update variants when a base character is clicked
    def show_variants(base_char):
        current_base[0] = base_char
        variants = generate_variants(base_char)
        
        variant_buttons = []
        for i, variant in enumerate(variants):
            btn = widgets.Button(
                description=variant,
                layout=widgets.Layout(width='40px', height='40px'),
                style=widgets.ButtonStyle(button_color='lightgreen' if i == 0 else None)
            )
            
            def on_variant_click(b, variant=variant):
                # Insert the selected variant at the cursor position
                cursor_pos = text_area.cursor_pos
                text = text_area.value
                
                # Add character at cursor position
                new_text = text[:cursor_pos] + variant + text[cursor_pos:]
                text_area.value = new_text
                
                # Move cursor after the inserted character
                text_area.cursor_pos = cursor_pos + len(variant)
                
                # Clear the variants
                variants_container.children = ()
                current_base[0] = None
            
            btn.on_click(on_variant_click)
            variant_buttons.append(btn)
        
        variants_container.children = tuple(variant_buttons)
    
    # Function for when a character button is clicked
    def on_char_click(b):
        base_char = b.description
        show_variants(base_char)
    
    # Assign click handlers to all character buttons
    for row in rows[:-1]:  # Skip the special row
        for button in row.children:
            if button is not space_button:  # Skip space button
                button.on_click(on_char_click)
    
    # Define special button behaviors
    def on_backspace_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        
        if cursor_pos > 0:
            # Delete the character before the cursor
            new_text = text[:cursor_pos-1] + text[cursor_pos:]
            text_area.value = new_text
            text_area.cursor_pos = cursor_pos - 1
        
        # Clear variants
        variants_container.children = ()
        current_base[0] = None
    
    def on_space_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        
        # Add space at cursor position
        new_text = text[:cursor_pos] + ' ' + text[cursor_pos:]
        text_area.value = new_text
        text_area.cursor_pos = cursor_pos + 1
        
        # Clear variants
        variants_container.children = ()
        current_base[0] = None
    
    backspace_button.on_click(on_backspace_click)
    space_button.on_click(on_space_click)
    
    # Add a label for the keyboard
    keyboard_label = widgets.HTML(
        value='<div style="text-align:center; margin-bottom:5px;"><b>Tigrigna Keyboard</b></div>'
    )
    
    # Create the final UI
    keyboard_container.children = [keyboard_label, text_area, variants_container] + rows
    
    # Create a button to transfer text to the spell checker
    transfer_button = widgets.Button(
        description='Send to Spell Checker',
        button_style='primary',
        icon='check',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    def on_transfer_click(b):
        # This function will later be connected to the spell checker
        print("Text to send to spell checker:", text_area.value)
        # Add code to transfer to spell checker later
        
    transfer_button.on_click(on_transfer_click)
    
    # Add button to keyboard container
    keyboard_container.children = list(keyboard_container.children) + [transfer_button]
    
    return keyboard_container, text_area

## Create and Display the Keyboard

In [ ]:
# Create and display the keyboard
keyboard, text_input = create_keyboard_ui()
display(keyboard)

## Integrate with Spell Checker

We can integrate this keyboard with the spell checker by using the text from the keyboard as input to the spell check function.

In [ ]:
# Function to load the dictionary
def load_dictionary(file_path='tigrigna_dictionary.txt'):
    """Load the Tigrigna dictionary"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            dictionary = {line.strip() for line in file if line.strip()}
        return dictionary
    except FileNotFoundError:
        print(f"Error: Dictionary file '{file_path}' not found.")
        return set()
    except Exception as e:
        print(f"Error loading dictionary: {e}")
        return set()

# Load the dictionary
dictionary = load_dictionary()
print(f"Loaded {len(dictionary)} words from the dictionary")

## Integrated Spell Checker UI with Keyboard

In [ ]:
def create_integrated_spell_checker_ui():
    """Create an integrated UI with keyboard and spell checker"""
    # Create the keyboard
    keyboard, text_input = create_keyboard_ui()
    
    # Output area for spell check results
    output_area = widgets.Output()
    
    # Create a spell check button
    spell_check_button = widgets.Button(
        description='Check Spelling',
        button_style='success',
        icon='check',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # Spell check function
    def check_spelling(text, dictionary):
        # Import functions from the spell checker notebook
        # Tokenize the text
        words = re.findall(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]+', text)
        words = [word.strip() for word in words if word.strip()]
        
        results = []
        for word in words:
            is_correct = word in dictionary
            results.append({
                'word': word,
                'is_correct': is_correct,
                'suggestions': [] if is_correct else get_suggestions(word, dictionary)
            })
        
        return results
    
    # Calculate Levenshtein distance for suggestions
    def levenshtein_distance(s1, s2):
        if len(s1) < len(s2):
            return levenshtein_distance(s2, s1)
        
        if len(s2) == 0:
            return len(s1)
        
        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        
        return previous_row[-1]
    
    # Get spelling suggestions
    def get_suggestions(word, dictionary, max_distance=2, max_suggestions=5):
        suggestions = []
        
        for dict_word in dictionary:
            distance = levenshtein_distance(word, dict_word)
            if distance <= max_distance:
                suggestions.append((dict_word, distance))
        
        suggestions.sort(key=lambda x: x[1])
        return [word for word, _ in suggestions[:max_suggestions]]
    
    # Format results as HTML
    def format_results(results):
        html = '<div style="font-family: Arial, sans-serif; line-height: 1.6;">'
        html += '<h3>Spell Check Results:</h3>'
        
        if not results:
            html += '<p>No words to check.</p>'
        else:
            html += '<ul style="list-style-type: none; padding-left: 0;">'
            
            for item in results:
                word = item['word']
                is_correct = item['is_correct']
                suggestions = item['suggestions']
                
                if is_correct:
                    html += f'<li style="margin-bottom: 10px;"><span style="color: green;">{word}</span> - Correct</li>'
                else:
                    html += f'<li style="margin-bottom: 10px;"><span style="color: red; text-decoration: underline;">{word}</span> - Misspelled'
                    
                    if suggestions:
                        html += '<ul style="margin-top: 5px; margin-bottom: 5px;">'
                        html += '<li>Suggestions:</li>'
                        for suggestion in suggestions:
                            html += f'<li style="margin-left: 20px;">{suggestion}</li>'
                        html += '</ul>'
                    else:
                        html += '<br>No suggestions available.'
                        
                    html += '</li>'
            
            html += '</ul>'
        
        html += '</div>'
        return html
    
    def on_check_spelling(b):
        text = text_input.value
        
        with output_area:
            output_area.clear_output()
            
            if not text.strip():
                display(HTML('<p style="color: orange;">Please enter some text to check.</p>'))
                return
            
            results = check_spelling(text, dictionary)
            display(HTML(format_results(results)))
    
    spell_check_button.on_click(on_check_spelling)
    
    # Update the transfer button to trigger spell checking
    keyboard.children[-1].on_click(on_check_spelling)
    
    # Create the integrated UI
    integrated_ui = widgets.VBox([
        widgets.HTML('<h1>Tigrigna Spelling Checker with Keyboard</h1>'),
        widgets.HTML('<p>Use the keyboard below to type Tigrigna text, then click "Check Spelling" to see results.</p>'),
        keyboard,
        spell_check_button,
        output_area
    ])
    
    return integrated_ui

In [ ]:
# Create and display the integrated UI
integrated_ui = create_integrated_spell_checker_ui()
display(integrated_ui)